# Spamku 
## May 4, 2020
## Version 2.0.1

### Import Libraries and Setup Variables

Import nltk libraries

Setup dictionaries for storing learning

In [7]:
# https://www.nltk.org/
import nltk;

# http://www.nltk.org/book/ch05.html
from nltk import pos_tag

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('cmudict')
from nltk.corpus import cmudict

# https://docs.python.org/3/library/os.html
import os

# https://docs.python.org/3/library/pickle.html
import pickle

# https://docs.python.org/3/library/collections.html
from collections import defaultdict


pos_dict=defaultdict(list)
# declare custom dictionary for determining part of speech
pronunciation_dict = cmudict.dict()

# https://docs.python.org/3/library/random.html
import random


import string



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\njayj\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\njayj\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package cmudict to
[nltk_data]     C:\Users\njayj\AppData\Roaming\nltk_data...
[nltk_data]   Package cmudict is already up-to-date!


### Read in Corpus of Sample Spam as a text file and also make a list of haiku, so each item in the list is a complete three line haiku

This file contains 8893 spam-ku


In [12]:
# the name of the corpus is text. It is just one long string, no newlines
filename="corpus/spamku.txt"
file = open(filename, mode = 'r')
text = ""
for line in file:
    text += line.strip()+' '
file.close()
c=0
l=[]
# haiku_list is the name of our list which holds the haiku in their original form
haiku_list=[]
file = open(filename, mode = 'r')
for line in file:
    c+=1
    l.append(line)
    if c==3:
        haiku_list.append(l)
        c=0
        l=[]
file.close()

### Read in pickle files 

pos_patterns is a dictionary with parts of speech patterns as keys, and how often that pattern occured in the corpus as values.  Each patterns is from one line of a haiku
For example: 'JJ NN NN' : 140  
  
pos_dict is a dictionary with indivdual parts of speech as keys, and the value is a list of all words from our spamku corpus that match that part of speech  
  
These dictionaries take a while to generate on my i5-3320M computer, so I pickled them.  See the end of the notebook to see how I generated them (those cells will consist of commented out code)



In [13]:
def loadData(filename): 
    file = open(filename, 'rb')      
    data = pickle.load(file) 
    file.close()
    return data

pos_patterns = loadData("pos_patterns.p")
pos_dict = loadData("pos_dict.p")

### Perform Markov Chain Analysis on Sample Poems

Determining the most likely word to follow each word in sample

In [14]:
def markov_chain(text):
    '''The input is a string of text and the output will be a dictionary with each word as
       a key and each value as the list of words that come after the key in the text.'''
    
    # Tokenize the text by word, though including punctuation
    words = text.split(' ')
    
    # Initialize a default dictionary to hold all of the words and next words
    m_dict = defaultdict(list)
    
    # Create a zipped list of all of the word pairs and put them in word: list of next words format
    for current_word, next_word in zip(words[0:-1], words[1:]):
        m_dict[current_word].append(next_word)

    # Convert the default dict back into a dictionary
    m_dict = dict(m_dict)
    return m_dict

mc=markov_chain(text)

def hormel_chain(text):
    '''The input is a string of text and the output will be a dictionary with each word as
       a key and each value as the list of words that come after the key in the text.'''
    
    # Tokenize the text by word, though including punctuation
    words = text.split(' ')
    
    # Initialize a default dictionary to hold all of the words and next words
    m_dict = [] #defaultdict(list)
    invalid = 0
    
    # Create a zipped list of all of the word pairs and put them in word: list of next words format
    for current_word, next_word in zip(words[0:-1], words[1:]):
        try:
            # print(current_word)
            current_word = current_word.lower().translate(str.maketrans('','',string.punctuation))
            count = get_syllable_count(current_word)
            pos = get_pos_label([current_word])
            m_dict[count][pos].append(next_word)
        except:
            invalid += 1
    # Convert the default dict back into a dictionary
    # m_dict = dict(m_dict)
    return m_dict

hc=hormel_chain(text)

print(hc)

[]


### Define functions to count syllables for each word

Testing with 3 words

In [15]:
def get_syllable_count(word):
    # print(pronunciation_dict[word])
    syl=[ x for x in pronunciation_dict[word][0] if x[-1].isdigit()]
    return len(syl)

print(get_syllable_count('pastrami'))
print(get_syllable_count('sausage'))
print(get_syllable_count('ribs'))


3
2
1


### Define function to return part of speech for each word

Testing with 3 words

In [16]:
def get_pos_label(word):
    return pos_tag(word)[0][1]

print(get_pos_label(['capicola']))
print(get_pos_label(['pastrami']))
print(get_pos_label(['andouille']))


NN
NN
NN


#### Create functions to choose POS pattern and syllable structure

In [21]:
pattern_prob = []
for k,v in pos_patterns.items():
    for i in range(v):
        try:
            if v != "":
                pattern_prob.append(k)
        except:
            print("error: ", k, v)
num_diff_patterns = len(pattern_prob)

syllable_lengths = {
    'VB': 1,
    'DT': 1,
    'RB': 2,
    'NN': 5,
    'VBG': 3,
    'IN': 1,
    'NNS': 4,
    'PRP$': 1,
    'CC': 1,
    'JJ': 2,
    'WP': 1,
    'VBZ': 1,
    'PRP': 1, 
    'VBN': 2,
    'JJS': 2,
    'MD': 1, 
    'TO': 1,
    'VBD': 2,
    'VBP': 2,
    'WRB': 1,
    'CD': 1, #cardinal digit, could be more
    'RBR': 2,
    'JJR': 2,
    'WDT': 1,
    'WP$': 1
}


# Maybe do a probability analysis on these?
# CC coordinating conjunction
# CD cardinal digit
# DT determiner
# EX existential there (like: “there is” … think of it like “there exists”)
# FW foreign word
# IN preposition/subordinating conjunction
# JJ adjective ‘big’
# JJR adjective, comparative ‘bigger’
# JJS adjective, superlative ‘biggest’
# LS list marker 1)
# MD modal could, will
# NN noun, singular ‘desk’
# NNS noun plural ‘desks’
# NNP proper noun, singular ‘Harrison’
# NNPS proper noun, plural ‘Americans’
# PDT predeterminer ‘all the kids’
# POS possessive ending parent’s
# PRP personal pronoun I, he, she
# PRP$ possessive pronoun my, his, hers
# RB adverb very, silently,
# RBR adverb, comparative better
# RBS adverb, superlative best
# RP particle give up
# TO, to go ‘to’ the store.
# UH interjection, errrrrrrrm
# VB verb, base form take
# VBD verb, past tense took
# VBG verb, gerund/present participle taking
# VBN verb, past participle taken
# VBP verb, sing. present, non-3d take
# VBZ verb, 3rd person sing. present takes
# WDT wh-determiner which
# WP wh-pronoun who, what
# WP$ possessive wh-pronoun whose
# WRB wh-abverb where, when

def choose_target_syllable_count(pattern, number_of_syllables_wanted):
    syllable_counts = []
    possible = False
    pattern_len = len(pattern)
    max_per_word = number_of_syllables_wanted - pattern_len
    for i in range(len(pattern)):
        while possible == False:
            x = randint(1, syllable_lengths[pattern[i]])
            if x <= max_per_word:
                possible = True
        syllable_counts.append[x]
        number_of_syllables_wanted -= x
        pattern_len -= 1
        max_per_word = number_of_syllables_wanted - pattern_len
    return syllable_counts
                    
        
        
        
    
    

### Process each line of sample

Determine the arrangement of syllables

Determine the parts of speech of each word

### Pick a pattern for each line

Choose a syllable sequence for the 1st, 2nd and 3rd line

In [24]:
from random import randint
line1 = random.choice(pattern_prob)
line2 = random.choice(pattern_prob)  # need to have two separate prob for 5 and 7
line3 = random.choice(pattern_prob)

l1=choose_target_syllable_count(line1, 5)
l2=choose_target_syllable_count(line1, 7)
l3=choose_target_syllable_count(line1, 5)



KeyError: 'I'

### Seed the poems 1st word of syllable length from pattern

In [ ]:
chain=markov_chain(text)

# Capitalize the first word
word1 = random.choice(list(chain.keys()))
# https://www.programiz.com/python-programming/methods/string/capitalize
sentence = word1.capitalize()


print(sentence)

### Use chain to pick next word with correct number of syllables

In [ ]:
line1text=""
word=""
c=0
for pos in line1:
    word=""
    possible = false
    try:
        while possible = false:
            word = random.choice(pos_dict[pos])
            if get_syllable_count(word) = l1[c]
                line1text += word " "
                c += 1
                possible = true
        except:
            print("Unexpected error when adding to dict:", sys.exc_info()[0])
            print("pos:", pos, l1[c], word)
            
        
        

In [ ]:
line1text

### Functions used to create pickled dictionaries

In [ ]:
# # Make pos patterns from haiku
# pos_patterns =  defaultdict(list)
# pos_patterns_markov_data = []
# haiku_pat = []
# for haiku in haiku_list:
#     for line in haiku:
#         l=""
#         line = line.strip()
#         for word in line.split():
#             word = clean_word(word)
#             try:
#                 l += pos_tag([word])[0][1]+" "
#             except:
#                 #I discovered that word is "" sometimes, which raises an error
#                 #print("error when building l", word)
#                 pass
#         haiku_pat.append(l)
#         if l in pos_patterns:
#             pos_patterns[l] += 1
#         else:
#             pos_patterns[l] = 1
#     pos_patterns_markov_data.append(l)

In [ ]:
# # Make pos_dict from spamku
#
# import string
# import sys
# c=0
# for word in text.split(' '):
#     c+=1
#     word = word.translate(str.maketrans('', '', string.punctuation))
#     word = word.lower()   
#     if c % 5000 == 0:
#         print(c, word)
#     try:
#         tok = pos_tag([word])
#     except:
#         pass
#         #I discovered that word is "" often which raises an error
# #         print("Unexpected error when using pos_tag:", sys.exc_info()[0])
# #         print("Word: ", word)
#     try:
#         pos_dict[tok[0][1]].append(tok[0][0])
#     except:
#         # Haven't seen any of these
#         print("Unexpected error when adding to dict:", sys.exc_info()[0])
#         print(tok)
# print('***************  Done  ***************')

In [ ]:
# pickle.dump(pos_dict, open("pos_dict.p", "wb"))
# pickle.dump(pos_patterns, open("pos_patterns.p", "wb"))